In [1]:
# 马尔可夫决策过程
import numpy as np

In [2]:
# 马尔可夫奖励过程
# 需要的变量
# 状态集合
# 状态转移概率（矩阵）
# 奖励函数
# 折扣因子

# 状态有6个
# 状态转移概率矩阵
P = [
    [0.9, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.6, 0.0, 0.4],
    [0.0, 0.0, 0.0, 0.0, 0.3, 0.7],
    [0.0, 0.2, 0.3, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
] 
P = np.array(P)

# 对应的奖励函数
rewards = [-1, -2, -2, 10, 1, 0]
# 折扣因子
gamma = 0.5  # 定义折扣因子

In [4]:
# 回报G的计算，给定一条序列,计算从某个索引（起始状态）开始到序列最后（终止状态）得到的回报
# G之间的递推关系
# Gt = Rt + gamma * G(t+1)
def compute_return(start_index,chain,gamma,rewards):
    G = 0
    # 所以这里是倒叙
    for i in reversed(range(start_index,len(chain))):
        G = gamma * G + rewards[chain[i] - 1]
    
    return G

In [7]:
# 计算回报G

# 序列
chain = [1,2,3,4,5,3,6]
start_index = 2
G = compute_return(start_index,chain,gamma,rewards)
print("根据本序列计算得到回报为：%s。" % G)

根据本序列计算得到回报为：3.0。


In [7]:
# 贝尔曼方程求解马尔可夫奖励的价值函数
def compute(P,rewards,gammma):
    ''' 利用贝尔曼方程的矩阵形式计算解析解,states_num是MRP的状态数 '''
    # V = R + gamma * PV
    # V = （I-gammaP）^-1R
    states_num = len(rewards)
    rewards = np.array(rewards).reshape((-1,1))
    temp = np.eye(states_num,states_num) # 生成num*num的对角阵
    temp = temp - gamma * P # [num,num]
    # 求矩阵的逆
    temp = np.linalg.inv(temp) # [num,num]
    # 矩阵乘法
    value = np.dot(temp,rewards)
    
    return value

In [12]:
# 计算价值函数
# 这个在确定的P,REWARDS,GAMMA下就是确定的，和具体的序列没有关系
V = compute(P, rewards, gamma)
print("MRP中每个状态价值分别为\n", V)

MRP中每个状态价值分别为
 [[-2.01950168]
 [-2.21451846]
 [ 1.16142785]
 [10.53809283]
 [ 3.58728554]
 [ 0.        ]]


In [3]:
# 马尔可夫决策过程的表示
# S,A,P,gamma,rewards
S = ["s1", "s2", "s3", "s4", "s5"]  # 状态集合
A = ["保持s1", "前往s1", "前往s2", "前往s3", "前往s4", "前往s5", "概率前往"]  # 动作集合

# 状态转移函数
# 格式：当前状态-采取的行动-转移的状态
P = {
    "s1-保持s1-s1": 1.0,
    "s1-前往s2-s2": 1.0,
    "s2-前往s1-s1": 1.0,
    "s2-前往s3-s3": 1.0,
    "s3-前往s4-s4": 1.0,
    "s3-前往s5-s5": 1.0,
    "s4-前往s5-s5": 1.0,
    "s4-概率前往-s2": 0.2,
    "s4-概率前往-s3": 0.4,
    "s4-概率前往-s4": 0.4,
}

# 奖励函数
# 格式：当前状态-采取行动
R = {
    "s1-保持s1": -1,
    "s1-前往s2": 0,
    "s2-前往s1": -1,
    "s2-前往s3": -2,
    "s3-前往s4": -2,
    "s3-前往s5": 0,
    "s4-前往s5": 10,
    "s4-概率前往": 1,
}
gamma = 0.5

# 一个马尔科夫决策过程
MDP = (S,A,P,R,gamma)

In [4]:
# 策略定义

# 1. 完全随机策略
# 即在每个状态下，智能体会以同样的概率选取它可能采取的动作。
Pi_1 = {
    "s1-保持s1": 0.5,
    "s1-前往s2": 0.5,
    "s2-前往s1": 0.5,
    "s2-前往s3": 0.5,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.5,
    "s4-概率前往": 0.5,
}

# 2.有一定设置的策略
Pi_2 = {
    "s1-保持s1": 0.6,
    "s1-前往s2": 0.4,
    "s2-前往s1": 0.3,
    "s2-前往s3": 0.7,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.1,
    "s4-概率前往": 0.9,
}

# util
def join(str1, str2):
    return str1 + '-' + str2

In [5]:
# 计算该 MDP 下，一个策略的状态价值函数。
# !!!转化前的 MDP 的状态价值函数和转化后的 MRP 的价值函数是一样的。!!!  ？？？
# 1. 将MDP中的action边缘化，转换为MRP
R_from_mdp_to_mrp = [-0.5, -1.5, -1.0, 5.5, 0] # 边缘化后的奖励函数
# 转化后的MRP的状态转移矩阵
P_from_mdp_to_mrp = [
    [0.5, 0.5, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.5, 0.5],
    [0.0, 0.1, 0.2, 0.2, 0.5],
    [0.0, 0.0, 0.0, 0.0, 1.0],
]
P_from_mdp_to_mrp = np.array(P_from_mdp_to_mrp)

In [8]:
# 2. 根据MRP计算value-state 的方法，计算,这个可以当中标准来对照其他方法
# 知道了value-state函数，就能够计算action-value函数
value = compute(P_from_mdp_to_mrp,R_from_mdp_to_mrp,gamma)
print("MDP中每个状态价值分别为\n", value)

MDP中每个状态价值分别为
 [[-1.22555411]
 [-1.67666232]
 [ 0.51890482]
 [ 6.0756193 ]
 [ 0.        ]]


In [11]:
# 蒙特卡洛方法
# 这是用来计算value-state函数
# 1. 采样
def sample(MDP,Pi,timestep_max,number):
    '''
    在策略pi的条件下，最长时间步为timestep_max,采样number组序列
    '''
    S, A, P, R, gamma = MDP
    episodes = []
    
    for _ in range(number):
        episode = []
        timestep = 0
        s = S[np.random.randint(4)] # 随机选择一个结束状态以外的状态作为起始状态
        
        # 沿着状态开始采样，当状态为终止状态或者时间步太长时,一次采样结束
        while s != 's5' and timestep <= timestep_max:
            timestep += 1
            
            # 根据策略选择状态s下的动作
            rand,temp = np.random.rand(),0
            # rand是随机一个概率
            # 感觉有点问题，但也能够保证随机性。。
            for a_opt in A:
                temp += Pi.get(join(s,a_opt),0)
                # 确定动作action
                if temp > rand:
                    a = a_opt
                    r = R.get(join(s,a),0)
                    break
            
            # 根据状态转移概率得到下一个状态
            rand,temp = np.random.rand(),0
            for s_opt in S:
                temp += P.get(join(join(s,a),s_opt),0)
                if temp > rand:
                    # 确定了下一个状态
                    s_next = s_opt
                    break
            
            # 至此，由s和pai，获得了 a，r还有下一个状态s_next
            episode.append((s,a,r,s_next)) 
            s = s_next
        episodes.append(episode)
    return episodes

In [24]:
# 计算state-value
def MC(episodes,V,N,gamma):
    '''
    episodes：采样的序列
    V：用于维护状态的总回报
    N：用于维护状态的计数器
    '''
    for episode in episodes:
        G = 0 # 回报，累计奖励
        # 为了增量更新，序列从后往前计算
        for i in range(len(episode)-1,-1,-1):
            (s,a,r,s_next) = episode[i]
            # 该时间步内的回报
            G = r + gamma*G
            N[s] = N[s] + 1 # 更新状态的计数
            V[s] = V[s] + (G - V[s])/N[s]     

In [31]:
# 采样
timestep_max = 20
episodes = sample(MDP,Pi_1,timestep_max,500)
gamma = 0.5
V = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}
N = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}
MC(episodes,V,N,gamma)

In [14]:
# 占用度量的计算
# 占用度量用来描述一个策略函数在MDP中的对不同的（s,a）访问概率
def occupancy(episodes,s,a,timesstep_max,gamma):
    ''' 计算状态动作对（s,a）出现的频率,以此来估算策略的占用度量 '''
    rho = 0
    total_times = np.zeros(timestep_max) # 记录每个时间步t各被经历过几次
    occur_times = np.zeros(timestep_max) # 记录(s_t,a_t)=(s,a)的次数，访问特定状态动作对的次数
    
    # 扫描所有的采样，然后进行状态动作对和时间步的计数
    for epi in episodes:
        for i in range(len(epi)):
            # 一个探索序列内
            (s_opt,a_opt,r,s_next) = epi[i]
            total_times[i] += 1   # 在
            
            if s == s_opt and a == a_opt:
                occur_times[i] += 1
    
    # 累加求和，计算占用度量
    for i in range(timestep_max):
        if total_times[i]:
            # 该时间步曾经出现过
            # 后半部分的表示有些不明白？？？
            rho += (gamma ** i) * (occur_times[i] / total_times[i])
    
    return (1 - gamma) * rho

In [22]:
gamma = 0.5
timestep_max = 1000
episodes_1 = sample(MDP, Pi_1, timestep_max, 1000) # 策略1的采样
episodes_2 = sample(MDP, Pi_2, timestep_max, 1000) # 策略2的采样

# 这里查看（s4,概率前往）动作对的情况
rho_1 = occupancy(episodes_1,"s4","概率前往",timestep_max, gamma)
rho_2 = occupancy(episodes_2,"s4","概率前往",timestep_max, gamma)
print(rho_1, rho_2)

0.10328751700335134 0.22613134269430213
